# Workflow for processing template Multi-timeline

In [1]:
import numpy as np
import sys, os
sys.path.append("../SRC")

import time
start_time = time.time()

#
# Define corners of field data.
# Order is SE, SW, NE, NW. Units are decimal degrees.
#
Corners = [-34.385141, 139.600725, 
           -34.385142, 139.599790, 
           -34.384350, 139.600920, 
           -34.384327, 139.599893] 

#
# Set up output directory
#
output = '../PNGS/TMP/'

#
# field_data is in the format: 'Field Site Name', 'Date'
#
field_data = ['BLA', '22APR18', '', 'Sentinel2a', 'UA_Panel', 'Radiance']

#
# CSV files with field site data (if available)
#
ls8_csvs = []
sent_csvs = []


#
# plot_scale defines the extent of the axes in the final two plots in the workflow.
# A scale from 0-1 means no reflectance to perfect reflectance. Typically, dark sites
# will range from 0-0.3, whereas bright sites will be between 0-1.
#
plot_scale = [0.0, 0.6, 0.0, 0.6]

In [2]:
import shutil

import matplotlib.pyplot as plt
import matplotlib

import datacube

from Site_Pipelines import *
from Multi_TimeLine import *

#
# Use notebook format (allows inline zooming and moving of figures)
# Set default font size for all plots
#
%matplotlib inline
matplotlib.rcParams.update({'font.size': 11})
matplotlib.rcParams['savefig.dpi'] = 300

#
# Remove old files in directory and create a new one
#
directory = os.path.dirname(output)
if os.path.exists(directory):
    shutil.rmtree(directory)
os.makedirs(directory)

In [3]:
#
# Use coordinates to source data on nearest operational rain gauge from the BoM
#
rain_dat = GetRainData.get_rain_data(Corners[0], Corners[1])

Nearest Rain Gauge Station -  3.95 km away

-------------------------------------------

         ID      Lat       Lon                            Name
2218  24564 -34.3515  139.6153  BLANCHETOWN LOCK 1            


In [4]:
#
# Colours used for plotting multi-coloured Lines
#
colpac=['#770000', '#FF0000', '#FF7700', '#FFFF00', '#77FF00', '#00FF00', 
        '#00FF77', '#00FFFF', '#0077FF', '#0000FF', '#000077', '#FF00FF', '#777777', '#770077', '#777700']

#
# Start Figure numbering at 1
#
fignum = 1

### Create DataFrames with GPS locations

In [5]:
import pandas as pd

def mkdf(Corners):
    temploc = pd.DataFrame([Corners[::2], Corners[1::2]], index=['Latitude', 'Longitude'])
    location = temploc.T.copy()
    return location

location = mkdf(Corners)

### Query Satellite data

Retrieve xarrays for satellite data, based on which satellite, query area and time.<BR>
sat_array will contain all band data<BR>
sat_bigarray will be a larger version of sat_array, which can be used for context plotting.

In [ ]:
ldc, sdc, lquery, squery, lquery2, squery2 = LongQuery.make_long_query(location)

ls8_array, s2a_array, s2b_array, ls8_bigarray, s2a_bigarray, s2b_bigarray = CreateLongArrays.create_long_arrays_nomask(ldc, sdc, lquery, squery, lquery2, squery2)

# Figure 

### Plot large-area context RGB array for Satellite data

In [ ]:
FIG_ManyLS8RGB.FIG_many_LS8RGB(ls8_bigarray, output, field_data, fignum)
fignum += 1

In [ ]:
FIG_ManyS2ARGB.FIG_many_S2ARGB(s2a_bigarray, output, field_data, fignum)
fignum += 1

In [ ]:
FIG_ManyS2BRGB.FIG_many_S2BRGB(s2b_bigarray, output, field_data, fignum)
fignum += 1

### Mask out clouds, based on thresholding

In [ ]:
ls8_daylist = CloudMask.cloud_mask(ls8_array, ls8_bigarray, 'LS8')
s2a_daylist = CloudMask.cloud_mask(s2a_array, s2a_bigarray, 'S2A')
s2b_daylist = CloudMask.cloud_mask(s2b_array, s2b_bigarray, 'S2B')

### Create a statistics dataframe for satellite data

In [ ]:
fls8_df, fs2a_df, fs2b_df = CreateManyStats.create_manystats(ls8_array, s2a_array, s2b_array, ls8_daylist, s2a_daylist, s2b_daylist, 'dummy', 'dummy', field_data)

# Figure

### Plot comparison spectra of satellite and field data, on a pixel-by-pixel basis

Error bars are shown for the satellite and field data, based on the standard deviation of the pixels
within the field. The black line shows the satellite data, which has been averaged over all pixels that satisfy the Datacube query ie. all pixels shown in Figure 11. The orange line shows satellite data which has been averaged over only those pixels that overlap with at least one field measurement.

In [ ]:
FIG_ManySatFieldBands.FIG_many_sat_field_bands(ls8_array, s2a_array, s2b_array, fls8_df, fs2a_df, fs2b_df, output, field_data, fignum)
fignum += 1

In [ ]:
FIG_MultiTimeLine.FIG_multi_time_line(fls8_df, fs2a_df, fs2b_df, ls8_csvs, sent_csvs, rain_dat, field_data, output, fignum)
fignum+=1

## How long did this notebook take to run?

In [ ]:
import datetime
print("This Notebook took ", str(datetime.timedelta(seconds=int((time.time() - start_time)))), "(h:m:s) to run")